In [1]:
using ReactiveMP
using StaticArrays
using BenchmarkTools
using Distributions

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1260
┌ Warning: Package ReactiveMP does not have Rocket in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rocket as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rocket into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.


In [2]:
function ArbitraryNode() 
    vars = @SVector [ nodevar(Symbol("x" * string(v))) for v in 1:5 ]
    fcn  = [ [ 1, 2 ], [ 3, 4 ], [5] ]
    return Node(+, vars, fcn) 
end

node = ArbitraryNode()

Node{typeof(+),5,Array{Array{Int64,1},1}}(NodeVariable[x1, x2, x3, x4, x5], [[1, 2], [3, 4], [5]])

In [13]:
AdditionNode() = Node(+, SA[ :in1, :in2, :out ], [ [ 1, 2, 3 ] ])
NormalNode() = Node(Normal, SA[ :mean, :precision, :value ], [ [ 1 ], [ 2 ], [ 3 ] ])

NormalNode (generic function with 1 method)

In [14]:
addnode = AdditionNode()

Node{typeof(+),3,Array{Array{Int64,1},1}}(NodeVariable[in1, in2, out], [[1, 2, 3]])

In [15]:
@btime deps($addnode, :out)

  120.733 ns (9 allocations: 368 bytes)


(NodeVariable[in1, in2], Array{NodeVariable,1}[])

In [7]:
nnode = NormalNode()

Node{UnionAll,3,Array{Array{Int64,1},1}}(NodeVariable[mean, precision, value], [[1], [2], [3]])

In [16]:
@btime deps($nnode, :value)

  210.694 ns (13 allocations: 592 bytes)


(NodeVariable[], Array{NodeVariable,1}[[mean], [precision]])

In [17]:
@code_warntype deps(addnode, :out)

Variables
  #self#::Core.Compiler.Const(ReactiveMP.deps, false)
  node::Node{typeof(+),3,Array{Array{Int64,1},1}}
  v::Symbol
  val::Array{Int64,1}
  #45::ReactiveMP.var"#45#47"{SArray{Tuple{3},NodeVariable,1,3}}
  #46::ReactiveMP.var"#46#48"{SArray{Tuple{3},NodeVariable,1,3}}
  vindex::Union{Nothing, Int64}
  cindex::Union{Nothing, Int64}
  vars::SArray{Tuple{3},NodeVariable,1,3}
  cls::Array{Array{Int64,1},1}
  factor::Array{Int64,1}
  vcindex::Union{Nothing, Int64}
  mdeps::Array{NodeVariable,1}
  clusterdeps::Array{Array{NodeVariable,1},1}

Body::Tuple{Array{NodeVariable,1},Array{Array{NodeVariable,1},1}}
1 ─       Core.NewvarNode(:(val))
│         Core.NewvarNode(:(#45))
│         Core.NewvarNode(:(#46))
│         Core.NewvarNode(:(vars))
│         Core.NewvarNode(:(cls))
│         Core.NewvarNode(:(factor))
│         Core.NewvarNode(:(vcindex))
│         Core.NewvarNode(:(mdeps))
│         Core.NewvarNode(:(clusterdeps))
│         (vindex = ReactiveMP.varindex(node, v))
│        